In [589]:
import pandas as pd


In [590]:
# Load the CSV file (replace the path with your own file location)
df = pd.read_excel('../final/danfoss_final_data_version.xlsx')
dfsanchuan = pd.read_excel('../final/sporlan_enemy_data_last_version.xlsx')

# Show the first 5 rows
must_have_data = ['Coil', 'Coil connection', 'Connection type',  'Electrical connection', 'Frequency [Hz]', 'Function', 'Refrigerants', 'Supply voltage [V] AC', ]
numeric = ['Connection size [in]', 'Cv value [gal/min]', 'Kv value [m³/h]', 'Max. Working Pressure [bar]', 'Max. Working Pressure [psig]', 'Orifice size [mm]', 'Orifice size fraction [in]',]

df[must_have_data] = df[must_have_data].fillna('BLANK').replace('', 'BLANK').replace(' ', 'BLANK')
df[numeric] = df[numeric].fillna('BLANK').replace('', 'BLANK').replace(' ', 'BLANK')

In [591]:
columns = df.columns.tolist()
print(columns)

# valve to target - search similar to this one
target_index = 0
target_index2 = 290
target_row = dfsanchuan.iloc[target_index]
print(target_row)
target_row2 = df.iloc[target_index2]
#df = df.drop(index=target_index)

# remove first 14 values
df = df.iloc[102:]
# print(len(df))

['<ID>', 'Product name', 'Actuator size', 'Actuator size [in]', 'Actuator system [mm]', 'Coil', 'Coil connection', 'Connection size [in]', 'Connection size [mm]', 'Connection type', 'Cv value [gal/min]', 'Cv value invert flow [gal/min]', 'Direction', 'Electrical connection', 'Enclosure rating IP', 'Frequency [Hz]', 'Function', 'Kv value [m³/h]', 'Kv value invert flow [m³/h]', 'Manual operation option', 'Max. Working Pressure [bar]', 'Max. Working Pressure [psig]', 'Media recommendations', 'Orifice size [mm]', 'Orifice size fraction [in]', 'Parts Program name', 'Power consumption [W]', 'Power consumption [W] 50Hz', 'Power consumption [W] 60Hz', 'Power consumption [W] DC', 'Refrigerants', 'Supply voltage [V] AC', 'Supply voltage [V] AC [max]', 'Supply voltage [V] AC [min]', 'Supply voltage [V] DC']
<ID>                                                                        A3 A3F1
Product name                                                         Solenoid valve
Actuator size           

In [592]:
#skippable
print(target_row)
print(target_row2)

<ID>                                                                        A3 A3F1
Product name                                                         Solenoid valve
Actuator size                                                                   NaN
Actuator size [in]                                                              NaN
Actuator system [mm]                                                            NaN
Coil                                                                MKC-1 (Class F)
Coil connection                                                Spade or DIN 43650-A
Connection size [in]                                                            1/4
Connection size [mm]                                                            NaN
Connection type                                                               Flare
Cv value [gal/min]                                                            1.801
Cv value invert flow [gal/min]                                              

In [593]:
import numpy as np

In [594]:

# 🧭 define your column groups

target_row = (
    target_row
    .fillna('BLANK')
    .replace('', 'BLANK')
    .replace(' ', 'BLANK')
)


def parse_numeric_value(x):
    """Parse numeric values with support for commas, dots, and fractions."""
    if x == "BLANK" or pd.isna(x):
        return None

    s = str(x).strip()

    # Handle fraction like "7/9"
    if "/" in s:
        try:
            num, denom = s.split("/")
            return float(num.replace(",", ".").strip()) / float(denom.replace(",", ".").strip())
        except Exception:
            return None

    # Handle decimal with comma or dot
    s = s.replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

In [595]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer('all-MiniLM-L6-v2')
THRESHOLD = 0.2

In [596]:
## TEXTUAL

def compute_similarity(target_value, candidate_value, model, THRESHOLD):
    """Compute similarity with exact match, blank handling, and semantic fallback."""

    t = target_value.strip().upper()
    c = candidate_value.strip().upper()

    # If either is blank → 0
    if is_blank(t) or is_blank(c):
        return 0.0

    # Exact match → 1.0
    if t == c:
        return 1.0

    # Else semantic similarity
    target_emb = model.encode([t])
    cand_emb = model.encode([c])
    sim = cosine_similarity(target_emb, cand_emb)[0][0]

    # Apply threshold to cut weak matches
    if sim < THRESHOLD:
        sim = 0.0

    return sim

def calculate_similarity_dict(danfoss_df, target_row, compare_columns):
    """Calculate similarity between one Sporlan product and all Danfoss rows, excluding blank-only rows."""


    # Identify non-blank columns for target
    non_blank_columns = [col for col in compare_columns if not is_blank(str(target_row[col]))]

    # If target has no non-blank fields → return 0 for non-blank Danfoss rows only
    if len(non_blank_columns) == 0:
        similarity_dict = {}
        for i in range(len(danfoss_df)):
            danfoss_blank = all(is_blank(str(danfoss_df.iloc[i][col])) for col in compare_columns)
            if not danfoss_blank:
                similarity_dict[str(danfoss_df.iloc[i]['<ID>'])] = 0.00
        return similarity_dict

    # Initialize similarity scores
    total_similarities = np.zeros(len(danfoss_df))

    for col in compare_columns:
        target_value = str(target_row[col]) if pd.notna(target_row[col]) else ''
        sims = []
        for i in range(len(danfoss_df)):
            candidate_value = str(danfoss_df.iloc[i][col]) if pd.notna(danfoss_df.iloc[i][col]) else ''
            sims.append(compute_similarity(target_value, candidate_value, model, THRESHOLD))
        total_similarities += np.array(sims)

    # Average over the number of non-blank target columns
    total_similarities /= len(non_blank_columns)

    # Build result dict excluding Danfoss rows with both columns blank
    similarity_dict = pd.Series(total_similarities, index=danfoss_df.index)
    return similarity_dict

def is_blank(value):
    """Check if a field is considered blank (empty string or 'BLANK' case-insensitive)."""
    return (not value) or (value.strip().upper() == 'BLANK')


In [597]:

similarity_results = calculate_similarity_dict(df, target_row, must_have_data)
top_matches = similarity_results.sort_values(ascending=False)
print(top_matches.head(32))

242    0.489002
134    0.482101
138    0.482101
137    0.482101
133    0.482101
120    0.481393
123    0.481393
119    0.481393
124    0.481393
116    0.481393
115    0.481393
114    0.481393
108    0.470352
392    0.455687
112    0.435560
111    0.435560
109    0.435560
128    0.406981
224    0.398224
132    0.391323
135    0.391323
136    0.391323
131    0.391323
118    0.390615
121    0.390615
122    0.390615
117    0.390615
130    0.390472
129    0.390472
127    0.390472
161    0.388938
148    0.388938
dtype: float64


In [598]:
## NUMERIC


#  Text column scoring
def calculate_match_percentages(dataframe, reference_row, columnss):
    exact_match = (dataframe[columnss] == reference_row[columnss])
    blank_match = (dataframe[columnss] == "BLANK") & (reference_row[columnss] != "BLANK")

    #  Score: exact = 1.0, blank match = 0.5
    column_scores = exact_match.astype(float) + (blank_match.astype(float) * 0.5)

    row_scores = column_scores.sum(axis=1)
    #  Row-level percentage
    return row_scores



def numeric_similarity(a, b):
    """Return similarity between two numeric values (0–1)."""
    a_val = parse_numeric_value(a)
    b_val = parse_numeric_value(b)

    if a_val is None or b_val is None:
        return 0.0

    return 1 - abs(a_val - b_val) / max(abs(a_val), abs(b_val), 1e-9)


def calculate_numeric_similarity(dataFrame, target, numeric_columns):
    """
    Calculate numeric similarity for each row against target_row.

    Args:
        dataFrame (pd.DataFrame): dataframe of records
        target (pd.Series): single target row
        numeric_columns (list): list of numeric column names

    Returns:
        pd.Series: numeric similarity score per row (0–100)
    """
    scores = []

    for i, row in dataFrame.iterrows():
        row_score = 0.0
        for col in numeric_columns:
            row_score += numeric_similarity(row[col], target[col])
        row_score /= len(numeric_columns)  # average per row
        scores.append(row_score * 100)     # convert to percentage

    return pd.Series(scores, index=dataFrame.index)

num_perc = calculate_numeric_similarity(df, target_row, numeric)
top_matches = num_perc.sort_values(ascending=False)
print(top_matches.head(20))


247    64.691064
363    64.053802
182    63.271247
362    62.625231
122    62.310111
245    62.310111
243    62.310111
123    62.310111
241    62.310111
299    62.310111
298    62.310111
113    62.310111
111    62.310111
297    62.310111
246    62.310111
250    62.310111
242    62.310111
253    62.310111
254    62.310111
244    62.310111
dtype: float64


In [599]:
## FINALFINALFINALFINALFINALFINALFINALFINAL
final_percentages = (num_perc +(similarity_results*100))/2
# calculate numeric scores for each column
df["final percentage"] = final_percentages
top_matches = final_percentages.sort_values(ascending=False)
top_indeces = top_matches.head(20).index
print(df.loc[top_indeces, ['<ID>', 'Product name', 'Function', "final percentage"]])

           <ID>    Product name Function  final percentage
242  032F809131  Solenoid valve       NC         55.605161
123  032F809631  Solenoid valve       NC         55.224698
124  032F809731  Solenoid valve       NC         54.510413
119  032F807631  Solenoid valve       NC         53.160911
111  032F809231  Solenoid valve       NC         52.933052
120  032F807731  Solenoid valve       NC         51.970435
133    032L8076  Solenoid valve       NC         51.747466
122    032F2127  Solenoid valve       NC         50.685792
134    032L8077  Solenoid valve       NC         50.556990
114  032F805931  Solenoid valve       NC         50.460206
137    032L8096  Solenoid valve       NC         50.283834
116  032F811831  Solenoid valve       NC         49.733920
138    032L8097  Solenoid valve       NC         49.569548
108  032F807331  Solenoid valve       NC         49.330921
241  032F212331  Solenoid valve       NC         49.326645
118    032F2087  Solenoid valve       NC         48.6220

In [600]:


matching_rows = df.copy()

# Print each row’s ID with its match %
for id_val, perc in zip(matching_rows['<ID>'], match_percentages):
    print(f"ID: {id_val} → {perc:.2f}% match")

ID: 032F9519 → 50.00% match
ID: 032F9513 → 50.00% match
ID: 032F9521 → 50.00% match
ID: 032F9514 → 50.00% match
ID: 032F9522 → 50.00% match
ID: 032F9515 → 50.00% match
ID: 032F807331 → 50.00% match
ID: 032F807431 → 50.00% match
ID: 032F8164 → 50.00% match
ID: 032F809231 → 50.00% match
ID: 032F810331 → 50.00% match
ID: 032F1216 → 50.00% match
ID: 032F805931 → 50.00% match
ID: 032F811931 → 50.00% match
ID: 032F811831 → 50.00% match
ID: 032F2086 → 50.00% match
ID: 032F2087 → 50.00% match
ID: 032F807631 → 50.00% match
ID: 032F807731 → 50.00% match
ID: 032F2116 → 0.00% match
ID: 032F2127 → 50.00% match
ID: 032F809631 → 50.00% match
ID: 032F809731 → 50.00% match
ID: 032F1255 → 50.00% match
ID: 032F1258 → 50.00% match
ID: 032F2016 → 50.00% match
ID: 032F2015 → 50.00% match
ID: 032F2047 → 50.00% match
ID: 032F2046 → 50.00% match
ID: 032L2086 → 50.00% match
ID: 032L2087 → 50.00% match
ID: 032L8076 → 50.00% match
ID: 032L8077 → 50.00% match
ID: 032L2116 → 0.00% match
ID: 032L2127 → 0.00% match
I

In [601]:
top_matches = match_percentages.sort_values(ascending=False)
top_indices = top_matches.head(10)
top_indices

171    100.0
172    100.0
267    100.0
174    100.0
230    100.0
281    100.0
268    100.0
232    100.0
192    100.0
193    100.0
dtype: float64

In [602]:

top_rows = df.loc[304]
print(top_rows)

<ID>                                                                       032F3130
Product name                                                         Solenoid valve
Actuator size                                                                   NaN
Actuator size [in]                                                              NaN
Actuator system [mm]                                                           13.5
Coil                                                                             No
Coil connection                                                               BLANK
Connection size [in]                                                          BLANK
Connection size [mm]                                                             16
Connection type                                                              Solder
Cv value [gal/min]                                                             3.01
Cv value invert flow [gal/min]                                              

In [603]:
selected_a = df_a.iloc[0].to_dict()
df_b["similarity"] = df_b.apply(lambda row: compute_similarity(selected_a, row.to_dict()), axis=1)
df_b_sorted = df_b.sort_values("similarity", ascending=False)

print("Top matches:\n", df_b_sorted[["id", "similarity"]])
best_match = df_b_sorted.iloc[0]
print("\n✅ Best match found:", best_match.to_dict())

NameError: name 'df_a' is not defined